# CUSP WASTE GRA
___
# 1 - Clean ACS datasets

For each sub-dataset from ACS, I keep only a few variables that I are then be of some use for the modeling:
1. Import the table
2. Perform some manipulation (e.g. columns sum)
3. Rename the columns
4. Keep a subset of the variables
5. Save it

In [2]:
import sys
import os
import pandas as pd
sys.path.append(os.getenv('WASTE') + '/wlib/')
import waste
import numpy as np

## A - Children

In [3]:
acs1 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Children/ACS_13_5YR_B09002_with_ann.csv')

In [4]:
acs1['hh_wchild'] = acs1['HD01_VD01'].convert_objects(convert_numeric=True) #Convert strings to numeric
acs1 = acs1[['GEO.id2', 'GEO.display-label', 'hh_wchild']] #Keep only a subset of variable
acs1 = acs1.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'}) #Rename some columns
#acs1.describe()

## B - Education

In [5]:
acs2 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Education/ACS_13_5YR_B15003_with_ann.csv')

In [6]:
cols = [c for c in acs2.columns if c != 'GEO.id2']
acs2[cols] = acs2[cols].convert_objects(convert_numeric=True)

In [7]:
# The first two lines define the columns I want to sum, and then sum them
cols = [c for c in acs2.columns if (c[2:4] == '01') and c[-2:] in ('20','21','22','23','24','25')]
acs2['college'] = acs2[cols].sum(axis=1)
acs2['ncollege'] =  acs2['HD01_VD01'] - acs2['college']

acs2 = acs2.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs2 = acs2[['tract_id', 'tract_name', 'college', 'ncollege']]
#acs2.describe()

## C - Employment

In [8]:
acs3 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Employment/ACS_13_5YR_B23025_with_ann.csv')

In [9]:
acs3[['empl', 'unempl']] = acs3[['HD01_VD04', 'HD01_VD05']].convert_objects(convert_numeric=True)
acs3 = acs3.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})

acs3 = acs3[['tract_id', 'tract_name', 'empl', 'unempl']]
#acs3.describe()

## D - Food stamps

In [10]:
acs4 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Food_Stamps/ACS_13_5YR_B19058_with_ann.csv')

In [11]:
acs4[['help', 'nhelp']] = acs4[['HD01_VD02', 'HD01_VD03']].convert_objects(convert_numeric=True)
acs4 = acs4.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})

acs4 = acs4[['tract_id', 'tract_name', 'help', 'nhelp']]
#acs4.describe()

## E - Income

In [12]:
acs5 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_HH_Income/ACS_13_5YR_B19001_with_ann.csv')

In [13]:
brackets = ['hh_tot', 'inc10', 'inc15', 'inc20', 'inc25', 'inc30', 'inc35', 'inc40', 'inc45',
            'inc50', 'inc60', 'inc75', 'inc100', 'inc125', 'inc150', 'inc200', 'inc200+']
cols = [c for c in acs5.columns if c[2:4] == '01']
acs5[brackets] = acs5[cols].convert_objects(convert_numeric=True)
acs5 = acs5.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})

acs5 = acs5[['tract_id', 'tract_name'] + brackets]
#acs5.describe()

## F - Household size

In [14]:
acs6 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_HH_size/ACS_13_5YR_B11016_with_ann.csv')

In [15]:
acs6[['family', 'nfamily', 'single']] = acs6[['HD01_VD02', 'HD01_VD09', 
                                              'HD01_VD10']].convert_objects(convert_numeric=True)
acs6['roomies'] = acs6['nfamily'] - acs6['single']

acs6 = acs6.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs6 = acs6[['tract_id', 'tract_name', 'family', 'single', 'roomies']]
#acs6.describe()

## G - Income

In [16]:
acs7 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Income/ACS_13_5YR_B19313_with_ann.csv')

In [17]:
acs7['aggr_inc'] = acs7['HD01_VD01'].convert_objects(convert_numeric=True)

acs7 = acs7.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs7 = acs7[['tract_id', 'tract_name', 'aggr_inc']]
#acs7.describe()

## H - Industry

In [18]:
acs8 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Industry/ACS_13_5YR_C24050_with_ann.csv')

In [19]:
ind = ['ind_tot', 'ind_mgt', 'ind_serv', 'ind_sale', 'ind_const', 'ind_trans']
cols = [c for c in acs8.columns if (c[2:4] == '01') and c[7:9] in ('01','16','30','44','58','72')]
acs8[ind] = acs8[cols].convert_objects(convert_numeric=True)

acs8 = acs8.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs8 = acs8[['tract_id', 'tract_name'] + ind]
#acs8.describe()

## I - Rooms

In [20]:
acs9 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Rooms/ACS_13_5YR_B25019_with_ann.csv')

In [21]:
acs9['rooms'] = acs9['HD01_VD01'].convert_objects(convert_numeric=True)
acs9 = acs9.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs9 = acs9[['tract_id', 'tract_name', 'rooms']]
#acs9.describe()

## J - Units

In [22]:
acs10 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Units/ACS_13_5YR_B25001_with_ann.csv')

In [23]:
acs10['units'] = acs10['HD01_VD01'].convert_objects(convert_numeric=True)
acs10 = acs10.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name'})
acs10 = acs10[['tract_id', 'tract_name', 'units']]
#acs10.describe()

## K - Age

In [24]:
acs11 = pd.read_csv(os.getenv('WASTE') + '/Data/CT_Age/ACS_13_5YR_S0101_with_ann.csv')

In [25]:
cols = [c for c in acs11.columns if c != 'GEO.id2']
acs11[cols] = acs11[cols].convert_objects(convert_numeric=True)
acs11 = acs11.rename(columns = {'GEO.id2': 'tract_id', 'GEO.display-label': 'tract_name',
                                'HC01_EST_VC01': 'pop13'})

cols_c = [c for c in acs11.columns if (c[2:6] == '01_E') and (c[-2:] in ('03', '04', '05', '06'))]
acs11['child'] = np.round(acs11[cols_c].sum(axis=1)*acs11['pop13']/100)

cols_a = [c for c in acs11.columns if (c[2:6] == '01_E') and c[-2:] in ('07', '08', '09', '10',
                                                                     '11', '12', '13', '14')]
acs11['adult'] = np.round(acs11[cols_a].sum(axis=1)*acs11['pop13']/100)

acs11['old'] = acs11['pop13'] - acs11['child'] - acs11['adult']

acs11 = acs11[['tract_id', 'tract_name', 'pop13', 'child', 'adult', 'old']]
#acs11.describe()

# 2 - Merge

In [26]:
acs = [acs1,acs2,acs3,acs4,acs5,acs6,acs7,acs8,acs9,acs10,acs11]
df = dict()
for i,d in enumerate(acs):
    if i==0:
        df[i] = acs1
    else:
        df[i] = pd.merge(df[i-1], d, on=['tract_id', 'tract_name'])

df[10].to_csv(os.getenv('WASTE') + '/Data/acs_m.csv')

In [27]:
desc = ['11-digit Census Tract id ', 'Census Tract Name',
       'Number of HH with children',
       'Number of persons with college education', 'Number of persons without college education',
       'Number of persons employed', 'Number of persons unemployed',
       'Number of households receiving food stamps or cash help',
       'Number of households not receiving food stamps or cash help',
       'Number of households', 'Number of households earning less than $10,000',
       'Number of households earning $10,000-$15,000', 'Number of households earning $15,000-$20,000',
       'Number of households earning $20,000-$25,000', 'Number of households earning $25,000-$30,000',
       'Number of households earning $30,000-$35,000', 'Number of households earning $35,000-$40,000',
       'Number of households earning $40,000-$45,000', 'Number of households earning $45,000-$50,000',
       'Number of households earning $50,000-$60,000', 'Number of households earning $60,000-$75,000',
       'Number of households earning $75,000-$100,000', 'Number of households earning $100,000-$125,000',
       'Number of households earning $125,000-$150,000', 'Number of households earning $150,000-$200,000',
       'Number of households earning more than $200,000',
       'Number of family households', 'Number of single-person households',
       'Number of non-family multipersons households',
       'Aggregated income', 'Number of persons working, all positions (equal to empl)',
       'Number of persons working, management', 'Number of persons working, services',
       'Number of persons working, sales', 'Number of persons working, construction',
       'Number of persons working, transportation',
       'Number of rooms', 'Number of units', 'Total population',
       'Number of persons under 19', 'Number of persons aged 19-59', 'Number of persons over 59']

metadata = dict()
for i,c in enumerate(df[10].columns.tolist()):
    metadata[c] = desc[i]
pd.DataFrame.from_dict(metadata,orient='index').to_csv(os.getenv('WASTE') + '/Data/acs_m_metadata.csv')